In [31]:
#Required packages
import tweepy
import argparse
import json
import ijson
import os
import re
import pandas as pd
#does not truncate columns when displaying
pd.set_option('display.max_colwidth', -1)
import time
import numpy
#set up working directory
os.getcwd()
os.chdir("/Users/nicolekelly/Documents/ft_aggregate/")

In [4]:
#Setting up Twitter API
#twitterkeys.py is a file that contains my consumer_key, consumer_secret, access_token and access_token_secret
import twitterkeys

auth = tweepy.OAuthHandler(twitterkeys.consumer_key, twitterkeys.consumer_secret)
auth.set_access_token(twitterkeys.access_token, twitterkeys.access_token_secret)

api = tweepy.API(auth)

In [32]:
#second method from David
#Pull in most recent 20 tweets
#need to figure out how to pull in nested dictionaries 
    #like retweeted_status (which gives more information than just an indicator for retweets), hashtags, etc
ArepaZone_timeline=api.user_timeline("ArepaZone")
arepa_tweets=[]
for tweet in ArepaZone_timeline:
    tweet_parts = {
        "id":tweet.id,
        "created_at":tweet.created_at,
        "text":tweet.text,
        #"retweeted_status":tweet.retweeted_status,
        "in_reply_to_status_id":tweet.in_reply_to_status_id,
        "in_reply_to_user_id":tweet.in_reply_to_user_id,
        "lang":tweet.lang,
        "source":tweet.source,
        "favorited":tweet.favorited,
        "coordinates":tweet.coordinates,
        "place":tweet.place,
        "geo":tweet.geo,
        "is_quote_status":tweet.is_quote_status,
        "contributors":tweet.contributors,
        "retweeted":tweet.retweeted,
        "truncated":tweet.truncated,
        "favorite_count":tweet.favorite_count,
        "retweet_count":tweet.retweet_count
    }
    arepa_tweets.append(tweet_parts)

ArepaZone_df = pd.DataFrame(arepa_tweets)
#ArepaZone_df.text

0     RT @VasiliaSD: Cheese cachapa from @ArepaZone. So delish even @177MilkStreet would approve. Thnx to @haden_andrew for the rec! My 👜is loade…
1     RT @foodrevoltblog: American History of Food Trucks: 1691-2017 https://t.co/Gk70gm7GyO                                                      
2     Attention Fairfax #ArepaLovers! In case you missed it, you can now order arepas via @UberEats!                                              
3     Food truck update! Truck is still in the shop but we hope to be back on the streets tomorrow or Thursday at the latest!                     
4     We're glad you enjoyed! We'll be offering it more often! https://t.co/EBLoDM9EfO                                                            
5     Great news! We are now live on @UberEATS in Fairfax, Virginia!                                                                              
6     Come visit us at @unionmarketdc and don't miss out! https://t.co/rPVpshPgID                                     

In [35]:
#Function does the same thing as above, make sure to have a df=tweets_to_datafram(values) for it to save the values
def tweets_to_dataframe(food_truck_twitter_handle, food_truck_twitter_handle_string):
    food_truck_twitter_handle_timeline=api.user_timeline('{}'.format(food_truck_twitter_handle_string))
    food_truck_twitter_handle_tweets=[]
    for tweet in food_truck_twitter_handle_timeline:
        tweet_parts = {
            "id":tweet.id,
            "created_at":tweet.created_at,
            "text":tweet.text,
            "in_reply_to_status_id":tweet.in_reply_to_status_id,
            "in_reply_to_user_id":tweet.in_reply_to_user_id,
            "lang":tweet.lang,
            "source":tweet.source,
            "favorited":tweet.favorited,
            "coordinates":tweet.coordinates,
            "place":tweet.place,
            "geo":tweet.geo,
            "is_quote_status":tweet.is_quote_status,
            "contributors":tweet.contributors,
            "retweeted":tweet.retweeted,
            "truncated":tweet.truncated,
            "favorite_count":tweet.favorite_count,
            "retweet_count":tweet.retweet_count
            }
        food_truck_twitter_handle_tweets.append(tweet_parts)
    food_truck_twitter_handle_df = pd.DataFrame(food_truck_twitter_handle_tweets)
    #When I append multiple food trucks together into a larger dataframe, I want to be able to identify them 
    food_truck_twitter_handle_df['username']='{}'.format(food_truck_twitter_handle_string)
    return food_truck_twitter_handle_df

In [36]:
captaincookiedc_df=tweets_to_dataframe('captaincookiedc','captaincookiedc')
captaincookiedc_df.head()

,contributors,coordinates,created_at,favorite_count,favorited,geo,id,in_reply_to_status_id,in_reply_to_user_id,is_quote_status,lang,place,retweet_count,retweeted,source,text,truncated,username
0,None,None,2017-02-05 18:04:05,0,False,None,828303193348141059,NaN,NaN,False,en,None,0,False,Twitter for iPhone,Guys. Guys. Super Bowl special: 2 assorted bakers' doz + 2 milks. Find it on @UberEATS or in store at 2000 Penn while it lasts! 🏈🏈🏈🍪🍪🍪,False,captaincookiedc
1,None,None,2017-02-03 19:45:21,0,False,None,827603902359076864,NaN,NaN,False,en,None,0,False,Twitter for iPad,Thanks State Dept! Happy Super Bowl weekend! PS: store is open til 2 if you need cookies,False,captaincookiedc
2,None,None,2017-02-03 15:45:37,2,False,None,827543574585737216,NaN,NaN,False,en,"Place(country='United States', place_type='city', country_code='US', name='Washington', bounding_box=BoundingBox(type='Polygon', _api=<tweepy.api.API object at 0x1126a72e8>, coordinates=[[[-77.119401, 38.801826], [-76.909396, 38.801826], [-76.909396, 38.9953797], [-77.119401, 38.9953797]]]), attributes={}, _api=<tweepy.api.API object at 0x1126a72e8>, contained_within=[], full_name='Washington, DC', id='01fbe706f872cb32', url='https://api.twitter.com/1.1/geo/id/01fbe706f872cb32.json')",0,False,Twitter for iPad,Cookie Monsters! Come meet us at L'Enfant Plaza! See you at 11:15,False,captaincookiedc
3,None,"{'type': 'Point', 'coordinates': [-77.02984504, 38.90240307]}",2017-02-03 15:14:35,0,False,"{'type': 'Point', 'coordinates': [38.90240307, -77.02984504]}",827535762765803520,NaN,NaN,False,en,"Place(country='United States', place_type='poi', country_code='US', name='Franklin Square Park', bounding_box=BoundingBox(type='Polygon', _api=<tweepy.api.API object at 0x1126a72e8>, coordinates=[[[-77.0307469367981, 38.90187120477314], [-77.0307469367981, 38.90187120477314], [-77.0307469367981, 38.90187120477314], [-77.0307469367981, 38.90187120477314]]]), attributes={}, _api=<tweepy.api.API object at 0x1126a72e8>, contained_within=[], full_name='Franklin Square Park', id='07d9f73736c80001', url='https://api.twitter.com/1.1/geo/id/07d9f73736c80001.json')",0,False,Twitter for iPad,I'm here at #Franklinsquare today for lunch until 2:00pm!,False,captaincookiedc
4,None,None,2017-02-03 14:57:00,1,False,None,827531340228784128,NaN,NaN,False,en,None,0,False,Twitter for iPad,"Greetings State Dept! We come in cookies (it's like peace, but better). Special cookie: Heath bar!",False,captaincookiedc


In [ ]:
#df_ needs to go first because some trucknames start with numbers
#I should probably get the loop working because this is a ton of lines for 200+ trucks
df_abunaifood=tweets_to_dataframe('abunaifood','abunaifood')
df_AmoriniPanini=tweets_to_dataframe('AmoriniPanini','AmoriniPanini')
df_AngryBurgerNOW=tweets_to_dataframe('AngryBurgerNOW','AngryBurgerNOW')
df_arepacrew=tweets_to_dataframe('arepacrew','arepacrew')
df_ArepaZone=tweets_to_dataframe('ArepaZone','ArepaZone')
df_AstroDoughnuts=tweets_to_dataframe('AstroDoughnuts','AstroDoughnuts')
df_Partytrucks=tweets_to_dataframe('Partytrucks','Partytrucks')
df_AZNeats=tweets_to_dataframe('AZNeats','AZNeats')
df_alocubanotruck=tweets_to_dataframe('alocubanotruck','alocubanotruck')
df_BabasBigBite=tweets_to_dataframe('BabasBigBite','BabasBigBite')
df_badabingdc=tweets_to_dataframe('badabingdc','badabingdc')
df_balkaniktaste=tweets_to_dataframe('balkaniktaste','balkaniktaste')
df_theballtruck=tweets_to_dataframe('theballtruck','theballtruck')
df_BanhMiAnnie=tweets_to_dataframe('BanhMiAnnie','BanhMiAnnie')
df_BasilThymeDC=tweets_to_dataframe('BasilThymeDC','BasilThymeDC')
df_bbqbusdc=tweets_to_dataframe('bbqbusdc','bbqbusdc')
df_gotbeachfries=tweets_to_dataframe('gotbeachfries','gotbeachfries')
df_BeirutDelights=tweets_to_dataframe('BeirutDelights','BeirutDelights')
df_bellavitawheels=tweets_to_dataframe('bellavitawheels','bellavitawheels')
df_bestburritos1=tweets_to_dataframe('bestburritos1','bestburritos1')
df_okbibija=tweets_to_dataframe('okbibija','okbibija')
df_bigcheesetruck=tweets_to_dataframe('bigcheesetruck','bigcheesetruck')
df_BITE2GO=tweets_to_dataframe('BITE2GO','BITE2GO')
df_eatBONMi=tweets_to_dataframe('eatBONMi','eatBONMi')
df_Borinquenlunchb=tweets_to_dataframe('Borinquenlunchb','Borinquenlunchb')
df_BLTfoodtruck=tweets_to_dataframe('BLTfoodtruck','BLTfoodtruck')
df_bratwurstking=tweets_to_dataframe('bratwurstking','bratwurstking')
df_bklynsandwichco=tweets_to_dataframe('bklynsandwichco','bklynsandwichco')
df_eatbrownbag=tweets_to_dataframe('eatbrownbag','eatbrownbag')
df_bubbleteaparty=tweets_to_dataframe('bubbleteaparty','bubbleteaparty')
df_burgorilla=tweets_to_dataframe('burgorilla','burgorilla')
df_CAslidercompany=tweets_to_dataframe('CAslidercompany','CAslidercompany')
df_cajunators=tweets_to_dataframe('cajunators','cajunators')
df_CapitalCW=tweets_to_dataframe('CapitalCW','CapitalCW')
df_capmacdc=tweets_to_dataframe('capmacdc','capmacdc')
df_CaptainCookieDC=tweets_to_dataframe('CaptainCookieDC','CaptainCookieDC')
df_Caribbean2Go=tweets_to_dataframe('Caribbean2Go','Caribbean2Go')
df_CarmensCarts=tweets_to_dataframe('CarmensCarts','CarmensCarts')
df_carnbbq=tweets_to_dataframe('carnbbq','carnbbq')
df_EatCarolinaQ=tweets_to_dataframe('EatCarolinaQ','EatCarolinaQ')
df_1styellowvendor=tweets_to_dataframe('1styellowvendor','1styellowvendor')
df_ChatpatTruck=tweets_to_dataframe('ChatpatTruck','ChatpatTruck')
df_cheesequakedc=tweets_to_dataframe('cheesequakedc','cheesequakedc')
df_chefalex2013=tweets_to_dataframe('chefalex2013','chefalex2013')
df_ChefonWheels11=tweets_to_dataframe('ChefonWheels11','ChefonWheels11')
#chefsebonwheels is deactivated currently
#df_chefsebonwheels=tweets_to_dataframe('chefsebonwheels','chefsebonwheels')
df_ChezAdilmos=tweets_to_dataframe('ChezAdilmos','ChezAdilmos')
df_chickfilamobile=tweets_to_dataframe('chickfilamobile','chickfilamobile')
df_ChittiGrill=tweets_to_dataframe('ChittiGrill','ChittiGrill')
df_ChixNStixDC=tweets_to_dataframe('ChixNStixDC','ChixNStixDC')
df_coles_palette=tweets_to_dataframe('coles_palette','coles_palette')
df_cornedbeefking=tweets_to_dataframe('cornedbeefking','cornedbeefking')
df_CrabCab=tweets_to_dataframe('CrabCab','CrabCab')
df_CRAVEIT2=tweets_to_dataframe('CRAVEIT2','CRAVEIT2')
df_cremedelacakedc=tweets_to_dataframe('cremedelacakedc','cremedelacakedc')
df_crepelovetruck=tweets_to_dataframe('crepelovetruck','crepelovetruck')
df_CrepesParfait=tweets_to_dataframe('CrepesParfait','CrepesParfait')
df_JoyCupcakes=tweets_to_dataframe('JoyCupcakes','JoyCupcakes')
df_CurbSideCrab=tweets_to_dataframe('CurbSideCrab','CurbSideCrab')
df_curbsidecupcake=tweets_to_dataframe('curbsidecupcake','curbsidecupcake')
df_CurleysQ=tweets_to_dataframe('CurleysQ','CurleysQ')
df_ThePieTruckDC=tweets_to_dataframe('ThePieTruckDC','ThePieTruckDC')
df_DcBallers=tweets_to_dataframe('DcBallers','DcBallers')
df_dc_doner=tweets_to_dataframe('dc_doner','dc_doner')
df_DCEmpanadas=tweets_to_dataframe('DCEmpanadas','DCEmpanadas')
df_dc_greekfood=tweets_to_dataframe('dc_greekfood','dc_greekfood')
df_dcpollos=tweets_to_dataframe('dcpollos','dcpollos')
df_DCQuesadilla=tweets_to_dataframe('DCQuesadilla','DCQuesadilla')
df_dcslices=tweets_to_dataframe('dcslices','dcslices')
df_DCSLIDERSTRUCK=tweets_to_dataframe('DCSLIDERSTRUCK','DCSLIDERSTRUCK')
df_DCTacoTruck=tweets_to_dataframe('DCTacoTruck','DCTacoTruck')
df_dirtysouthdeli=tweets_to_dataframe('dirtysouthdeli','dirtysouthdeli')
df_DolciGelatiTruk=tweets_to_dataframe('DolciGelatiTruk','DolciGelatiTruk')
df_donburidc=tweets_to_dataframe('donburidc','donburidc')
df_DorothyMoon1=tweets_to_dataframe('DorothyMoon1','DorothyMoon1')
df_dougthefooddude=tweets_to_dataframe('dougthefooddude','dougthefooddude')
df_DuckysGrub=tweets_to_dataframe('DuckysGrub','DuckysGrub')
df_elFuegoDc=tweets_to_dataframe('elFuegoDc','elFuegoDc')
df_EndlessPLLC=tweets_to_dataframe('EndlessPLLC','EndlessPLLC')
df_FarEastTG=tweets_to_dataframe('FarEastTG','FarEastTG')
df_Fasikacuisin=tweets_to_dataframe('Fasikacuisin','Fasikacuisin')
df_FavaPot=tweets_to_dataframe('FavaPot','FavaPot')
df_FedCityBros=tweets_to_dataframe('FedCityBros','FedCityBros')
df_feelincrabby=tweets_to_dataframe('feelincrabby','feelincrabby')
df_eatFireNRice=tweets_to_dataframe('eatFireNRice','eatFireNRice')
df_firemanscafe=tweets_to_dataframe('firemanscafe','firemanscafe')
df_FoodLady2011=tweets_to_dataframe('FoodLady2011','FoodLady2011')
df_foodforceone1=tweets_to_dataframe('foodforceone1','foodforceone1')
df_freshgreenfood=tweets_to_dataframe('freshgreenfood','freshgreenfood')
df_FroZenYoToGo=tweets_to_dataframe('FroZenYoToGo','FroZenYoToGo')
df_FCFoodTruck=tweets_to_dataframe('FCFoodTruck','FCFoodTruck')
df_GeorgesWingTruk=tweets_to_dataframe('GeorgesWingTruk','GeorgesWingTruk')
df_gofishtruckdc=tweets_to_dataframe('gofishtruckdc','gofishtruckdc')
df_mobilekitchen=tweets_to_dataframe('mobilekitchen','mobilekitchen')
df_GAHDTruck=tweets_to_dataframe('GAHDTruck','GAHDTruck')
df_GREENEGGBURGERS=tweets_to_dataframe('GREENEGGBURGERS','GREENEGGBURGERS')
df_GridsWaffles=tweets_to_dataframe('GridsWaffles','GridsWaffles')
df_GuaposFoodTruck=tweets_to_dataframe('GuaposFoodTruck','GuaposFoodTruck')
df_habebe19=tweets_to_dataframe('habebe19','habebe19')
df_wherehalal=tweets_to_dataframe('wherehalal','wherehalal')
df_hardysbbqdivine=tweets_to_dataframe('hardysbbqdivine','hardysbbqdivine')
df_healthyfool=tweets_to_dataframe('healthyfool','healthyfool')
df_henhousedc=tweets_to_dataframe('henhousedc','henhousedc')
df_HolyCrepesTruck=tweets_to_dataframe('HolyCrepesTruck','HolyCrepesTruck')
df_HotPeopleFood=tweets_to_dataframe('HotPeopleFood','HotPeopleFood')
df_DCHotWheels=tweets_to_dataframe('DCHotWheels','DCHotWheels')
df_HouseofFalafel=tweets_to_dataframe('HouseofFalafel','HouseofFalafel')
df_hulagirltruck=tweets_to_dataframe('hulagirltruck','hulagirltruck')
df_hungryheartllc=tweets_to_dataframe('hungryheartllc','hungryheartllc')
df_italiansubs1=tweets_to_dataframe('italiansubs1','italiansubs1')
df_JMCCurbside=tweets_to_dataframe('JMCCurbside','JMCCurbside')
df_jerkchickenfest=tweets_to_dataframe('jerkchickenfest','jerkchickenfest')
df_KababjiTruck=tweets_to_dataframe('KababjiTruck','KababjiTruck')
df_KabobBites=tweets_to_dataframe('KabobBites','KabobBites')
df_kabob_king=tweets_to_dataframe('kabob_king','kabob_king')
df_kabobpalace1=tweets_to_dataframe('kabobpalace1','kabobpalace1')
df_KaftaMania=tweets_to_dataframe('KaftaMania','KaftaMania')
df_kalaverasdc=tweets_to_dataframe('kalaverasdc','kalaverasdc')
df_KimchiBBQ=tweets_to_dataframe('KimchiBBQ','KimchiBBQ')
df_dckbbqbox=tweets_to_dataframe('dckbbqbox','dckbbqbox')
df_korengy=tweets_to_dataframe('korengy','korengy')
df_kravingkabob=tweets_to_dataframe('kravingkabob','kravingkabob')
df_kushimototruck=tweets_to_dataframe('kushimototruck','kushimototruck')
df_LATacoTruck=tweets_to_dataframe('LATacoTruck','LATacoTruck')
df_LaTingeriaTruck=tweets_to_dataframe('LaTingeriaTruck','LaTingeriaTruck')
df_latinamericanf1=tweets_to_dataframe('latinamericanf1','latinamericanf1')
df_lemongrasstruck=tweets_to_dataframe('lemongrasstruck','lemongrasstruck')
df_LilypadontheRun=tweets_to_dataframe('LilypadontheRun','LilypadontheRun')
df_Limetreetruck=tweets_to_dataframe('Limetreetruck','Limetreetruck')
df_LLuncheonette=tweets_to_dataframe('LLuncheonette','LLuncheonette')
df_LittleItalyDC=tweets_to_dataframe('LittleItalyDC','LittleItalyDC')
df_chefty99=tweets_to_dataframe('chefty99','chefty99')
df_loslobosburrito=tweets_to_dataframe('loslobosburrito','loslobosburrito')
df_macattackdc=tweets_to_dataframe('macattackdc','macattackdc')
df_macsdonuts=tweets_to_dataframe('macsdonuts','macsdonuts')
df_MakiShopTruck=tweets_to_dataframe('MakiShopTruck','MakiShopTruck')
df_MamasDonutBites=tweets_to_dataframe('MamasDonutBites','MamasDonutBites')
df_MayurKababHouse=tweets_to_dataframe('MayurKababHouse','MayurKababHouse')
df_Med_Delight=tweets_to_dataframe('Med_Delight','Med_Delight')
df_Meggrollmania=tweets_to_dataframe('Meggrollmania','Meggrollmania')
df_meskihealthy2go=tweets_to_dataframe('meskihealthy2go','meskihealthy2go')
df_mesobonwheels=tweets_to_dataframe('mesobonwheels','mesobonwheels')
df_Miamiviceburger=tweets_to_dataframe('Miamiviceburger','Miamiviceburger')
df_MidniteCC=tweets_to_dataframe('MidniteCC','MidniteCC')
df_MightyDogAcai=tweets_to_dataframe('MightyDogAcai','MightyDogAcai')
df_MisoHoneyTruck=tweets_to_dataframe('MisoHoneyTruck','MisoHoneyTruck')
df_mohmohdumpling=tweets_to_dataframe('mohmohdumpling','mohmohdumpling')
df_mojaitalatinfla=tweets_to_dataframe('mojaitalatinfla','mojaitalatinfla')
df_Mojotruck=tweets_to_dataframe('Mojotruck','Mojotruck')
df_Miyagifoodtruck=tweets_to_dataframe('Miyagifoodtruck','Miyagifoodtruck')
df_MuncheezDC=tweets_to_dataframe('MuncheezDC','MuncheezDC')
#NaanStopDC is currently deactivated
#df_NaanStopDC=tweets_to_dataframe('NaanStopDC','NaanStopDC')
df_NeatMeatDC=tweets_to_dataframe('NeatMeatDC','NeatMeatDC')
df_NYDeliTruck=tweets_to_dataframe('NYDeliTruck','NYDeliTruck')
df_OliviasCupcakes=tweets_to_dataframe('OliviasCupcakes','OliviasCupcakes')
df_OohDaTChickeN=tweets_to_dataframe('OohDaTChickeN','OohDaTChickeN')
df_orangecowdc=tweets_to_dataframe('orangecowdc','orangecowdc')
df_overtherice=tweets_to_dataframe('overtherice','overtherice')
df_parskabob=tweets_to_dataframe('parskabob','parskabob')
df_passl0nfrult=tweets_to_dataframe('passl0nfrult','passl0nfrult')
df_pepefoodtruck=tweets_to_dataframe('pepefoodtruck','pepefoodtruck')
df_perubrothers=tweets_to_dataframe('perubrothers','perubrothers')
df_buritosontherun=tweets_to_dataframe('buritosontherun','buritosontherun')
df_philliesphamous=tweets_to_dataframe('philliesphamous','philliesphamous')
df_phojunkies=tweets_to_dataframe('phojunkies','phojunkies')
#PhoBachi changed its handle to @PhoBachi
#df_PhoBachiInc=tweets_to_dataframe('PhoBachiInc','PhoBachiInc')
df_PhoBachi=tweets_to_dataframe('PhoBachi','PhoBachi')
df_dcphonation=tweets_to_dataframe('dcphonation','dcphonation')
df_PhoWheels=tweets_to_dataframe('PhoWheels','PhoWheels')
df_pleasantpops=tweets_to_dataframe('pleasantpops','pleasantpops')
df_PoppedRepublic=tweets_to_dataframe('PoppedRepublic','PoppedRepublic')
df_dcpuddin=tweets_to_dataframe('dcpuddin','dcpuddin')
df_quickwraps=tweets_to_dataframe('quickwraps','quickwraps')
df_therandyradish=tweets_to_dataframe('therandyradish','therandyradish')
df_RebasFunnelCake=tweets_to_dataframe('RebasFunnelCake','RebasFunnelCake')
df_LobstertruckDC=tweets_to_dataframe('LobstertruckDC','LobstertruckDC')
df_redbonefc=tweets_to_dataframe('redbonefc','redbonefc')
df_ReggaeVibesTruk=tweets_to_dataframe('ReggaeVibesTruk','ReggaeVibesTruk')
df_riochurrascoDC=tweets_to_dataframe('riochurrascoDC','riochurrascoDC')
df_RitoLoco=tweets_to_dataframe('RitoLoco','RitoLoco')
df_RoamingRoti=tweets_to_dataframe('RoamingRoti','RoamingRoti')
df_RocklandsTruck=tweets_to_dataframe('RocklandsTruck','RocklandsTruck')
df_RockSalt_=tweets_to_dataframe('RockSalt_','RockSalt_')
df_Rollinpizzadc=tweets_to_dataframe('Rollinpizzadc','Rollinpizzadc')
df_rollinkabob=tweets_to_dataframe('rollinkabob','rollinkabob')
df_RollsOnRolls=tweets_to_dataframe('RollsOnRolls','RollsOnRolls')
df_RovingItalian=tweets_to_dataframe('RovingItalian','RovingItalian')
df_asgharboa=tweets_to_dataframe('asgharboa','asgharboa')
df_saffrontruckdc=tweets_to_dataframe('saffrontruckdc','saffrontruckdc')
df_SangOnWheels=tweets_to_dataframe('SangOnWheels','SangOnWheels')
df_SaransVegiTruck=tweets_to_dataframe('SaransVegiTruck','SaransVegiTruck')
df_SateTruck=tweets_to_dataframe('SateTruck','SateTruck')
df_Scoops2u=tweets_to_dataframe('Scoops2u','Scoops2u')
#Senortaco7 appears deactivated
#df_senortaco7=tweets_to_dataframe('senortaco7','senortaco7')
df_SeoulFoodDC=tweets_to_dataframe('SeoulFoodDC','SeoulFoodDC')
#Sidewalkssweets tweets are protected, I could follow them but for now I will ignore
#df_sidewalksweets=tweets_to_dataframe('sidewalksweets','sidewalksweets')
df_simple_wheels=tweets_to_dataframe('simple_wheels','simple_wheels')
df_simplysoulfood=tweets_to_dataframe('simplysoulfood','simplysoulfood')
df_Sinplicity1=tweets_to_dataframe('Sinplicity1','Sinplicity1')
df_SloppyMamas=tweets_to_dataframe('SloppyMamas','SloppyMamas')
df_BayouGators=tweets_to_dataframe('BayouGators','BayouGators')
df_smokingkowbbq=tweets_to_dataframe('smokingkowbbq','smokingkowbbq')
df_solmexicangrill=tweets_to_dataframe('solmexicangrill','solmexicangrill')
df_SouthMeetsEast=tweets_to_dataframe('SouthMeetsEast','SouthMeetsEast')
df_SouvlakiStop=tweets_to_dataframe('SouvlakiStop','SouvlakiStop')
df_SpitfireTruck=tweets_to_dataframe('SpitfireTruck','SpitfireTruck')
df_steakbites=tweets_to_dataframe('steakbites','steakbites')
df_stellaspopkern=tweets_to_dataframe('stellaspopkern','stellaspopkern')
#StreetCream34 seems deactivated
#df_StreetCream34=tweets_to_dataframe('StreetCream34','StreetCream34')
#Sundevich made it to the big time (aka multiple storefronts), no longer a truck
#df_SUNdeVICHtruck=tweets_to_dataframe('SUNdeVICHtruck','SUNdeVICHtruck')
df_surfsidetruckdc=tweets_to_dataframe('surfsidetruckdc','surfsidetruckdc')
df_SUSHIPAO=tweets_to_dataframe('SUSHIPAO','SUSHIPAO')
df_sweetbitestruck=tweets_to_dataframe('sweetbitestruck','sweetbitestruck')
df_swizzlerfoods=tweets_to_dataframe('swizzlerfoods','swizzlerfoods')
df_takorean=tweets_to_dataframe('takorean','takorean')
df_tapastruckdc=tweets_to_dataframe('tapastruckdc','tapastruckdc')
df_tashascookies=tweets_to_dataframe('tashascookies','tashascookies')
df_TasteOfEasternE=tweets_to_dataframe('TasteOfEasternE','TasteOfEasternE')
df_TastyFriedDC=tweets_to_dataframe('TastyFriedDC','TastyFriedDC')
df_tastykabob=tweets_to_dataframe('tastykabob','tastykabob')
df_TastyToranj=tweets_to_dataframe('TastyToranj','TastyToranj')
df_TempoDiPasta=tweets_to_dataframe('TempoDiPasta','TempoDiPasta')
df_thaimachine=tweets_to_dataframe('thaimachine','thaimachine')
df_cheesecaketruc=tweets_to_dataframe('cheesecaketruc','cheesecaketruc')
df_thecornfactory=tweets_to_dataframe('thecornfactory','thecornfactory')
df_tfmcupcakery=tweets_to_dataframe('tfmcupcakery','tfmcupcakery')
df_tiscreamery=tweets_to_dataframe('tiscreamery','tiscreamery')
df_thewagonmeal=tweets_to_dataframe('thewagonmeal','thewagonmeal')
df_TinHeavenDC=tweets_to_dataframe('TinHeavenDC','TinHeavenDC')
df_tokyointhecity=tweets_to_dataframe('tokyointhecity','tokyointhecity')
df_topdogtruck=tweets_to_dataframe('topdogtruck','topdogtruck')
df_topstrucks=tweets_to_dataframe('topstrucks','topstrucks')
df_tortugatruck=tweets_to_dataframe('tortugatruck','tortugatruck')
df_BakedTSR=tweets_to_dataframe('BakedTSR','BakedTSR')
df_TURKISHKABOBDC=tweets_to_dataframe('TURKISHKABOBDC','TURKISHKABOBDC')
df_urbanbumpkinbbq=tweets_to_dataframe('urbanbumpkinbbq','urbanbumpkinbbq')
df_urbanpoutine=tweets_to_dataframe('urbanpoutine','urbanpoutine')
df_VillageCafeINC=tweets_to_dataframe('VillageCafeINC','VillageCafeINC')
df_wassubdc=tweets_to_dataframe('wassubdc','wassubdc')
df_WestraysFinest=tweets_to_dataframe('WestraysFinest','WestraysFinest')
df_whatthephodc=tweets_to_dataframe('whatthephodc','whatthephodc')
df_wijammincaters=tweets_to_dataframe('wijammincaters','wijammincaters')
#rebranded as @NuVeganCafe, as of 2/1/16 the tweeted that they plan to bring back the truck this month
#df_WoodlandsVB=tweets_to_dataframe('WoodlandsVB','WoodlandsVB')
df_YellaFoodTruck=tweets_to_dataframe('YellaFoodTruck','YellaFoodTruck')
df_yellowvendor=tweets_to_dataframe('yellowvendor','yellowvendor')
df_YummyYum_Food=tweets_to_dataframe('YummyYum_Food','YummyYum_Food')
df_yumplingdc=tweets_to_dataframe('yumplingdc','yumplingdc')
#ZBurgerMobile seems to be deactivated
#df_ZBurgerMobile=tweets_to_dataframe('ZBurgerMobile','ZBurgerMobile')
df_ZestyKabob=tweets_to_dataframe('ZestyKabob','ZestyKabob')

In [52]:
#Combine all food trucks into one dataset
combined_truck_df=df_abunaifood
combined_truck_df=combined_truck_df.append(df_AmoriniPanini)
combined_truck_df=combined_truck_df.append(df_AngryBurgerNOW)
combined_truck_df=combined_truck_df.append(df_arepacrew)
combined_truck_df=combined_truck_df.append(df_ArepaZone)
combined_truck_df=combined_truck_df.append(df_AstroDoughnuts)
combined_truck_df=combined_truck_df.append(df_Partytrucks)
combined_truck_df=combined_truck_df.append(df_AZNeats)
combined_truck_df=combined_truck_df.append(df_alocubanotruck)
combined_truck_df=combined_truck_df.append(df_BabasBigBite)
combined_truck_df=combined_truck_df.append(df_badabingdc)
combined_truck_df=combined_truck_df.append(df_balkaniktaste)
combined_truck_df=combined_truck_df.append(df_theballtruck)
combined_truck_df=combined_truck_df.append(df_BanhMiAnnie)
combined_truck_df=combined_truck_df.append(df_BasilThymeDC)
combined_truck_df=combined_truck_df.append(df_bbqbusdc)
combined_truck_df=combined_truck_df.append(df_gotbeachfries)
combined_truck_df=combined_truck_df.append(df_BeirutDelights)
combined_truck_df=combined_truck_df.append(df_bellavitawheels)
combined_truck_df=combined_truck_df.append(df_bestburritos1)
combined_truck_df=combined_truck_df.append(df_okbibija)
combined_truck_df=combined_truck_df.append(df_bigcheesetruck)
combined_truck_df=combined_truck_df.append(df_BITE2GO)
combined_truck_df=combined_truck_df.append(df_eatBONMi)
combined_truck_df=combined_truck_df.append(df_Borinquenlunchb)
combined_truck_df=combined_truck_df.append(df_BLTfoodtruck)
combined_truck_df=combined_truck_df.append(df_bratwurstking)
combined_truck_df=combined_truck_df.append(df_bklynsandwichco)
combined_truck_df=combined_truck_df.append(df_eatbrownbag)
combined_truck_df=combined_truck_df.append(df_bubbleteaparty)
combined_truck_df=combined_truck_df.append(df_burgorilla)
combined_truck_df=combined_truck_df.append(df_CAslidercompany)
combined_truck_df=combined_truck_df.append(df_cajunators)
combined_truck_df=combined_truck_df.append(df_CapitalCW)
combined_truck_df=combined_truck_df.append(df_capmacdc)
combined_truck_df=combined_truck_df.append(df_CaptainCookieDC)
combined_truck_df=combined_truck_df.append(df_Caribbean2Go)
combined_truck_df=combined_truck_df.append(df_CarmensCarts)
combined_truck_df=combined_truck_df.append(df_carnbbq)
combined_truck_df=combined_truck_df.append(df_EatCarolinaQ)
combined_truck_df=combined_truck_df.append(df_1styellowvendor)
combined_truck_df=combined_truck_df.append(df_ChatpatTruck)
combined_truck_df=combined_truck_df.append(df_cheesequakedc)
combined_truck_df=combined_truck_df.append(df_chefalex2013)
combined_truck_df=combined_truck_df.append(df_ChefonWheels11)
combined_truck_df=combined_truck_df.append(df_ChezAdilmos)
combined_truck_df=combined_truck_df.append(df_chickfilamobile)
combined_truck_df=combined_truck_df.append(df_ChittiGrill)
combined_truck_df=combined_truck_df.append(df_ChixNStixDC)
combined_truck_df=combined_truck_df.append(df_coles_palette)
combined_truck_df=combined_truck_df.append(df_cornedbeefking)
combined_truck_df=combined_truck_df.append(df_CrabCab)
combined_truck_df=combined_truck_df.append(df_CRAVEIT2)
combined_truck_df=combined_truck_df.append(df_cremedelacakedc)
combined_truck_df=combined_truck_df.append(df_crepelovetruck)
combined_truck_df=combined_truck_df.append(df_CrepesParfait)
combined_truck_df=combined_truck_df.append(df_JoyCupcakes)
combined_truck_df=combined_truck_df.append(df_CurbSideCrab)
combined_truck_df=combined_truck_df.append(df_curbsidecupcake)
combined_truck_df=combined_truck_df.append(df_CurleysQ)
combined_truck_df=combined_truck_df.append(df_ThePieTruckDC)
combined_truck_df=combined_truck_df.append(df_DcBallers)
combined_truck_df=combined_truck_df.append(df_dc_doner)
combined_truck_df=combined_truck_df.append(df_DCEmpanadas)
combined_truck_df=combined_truck_df.append(df_dc_greekfood)
combined_truck_df=combined_truck_df.append(df_dcpollos)
combined_truck_df=combined_truck_df.append(df_DCQuesadilla)
combined_truck_df=combined_truck_df.append(df_dcslices)
combined_truck_df=combined_truck_df.append(df_DCSLIDERSTRUCK)
combined_truck_df=combined_truck_df.append(df_DCTacoTruck)
combined_truck_df=combined_truck_df.append(df_dirtysouthdeli)
combined_truck_df=combined_truck_df.append(df_DolciGelatiTruk)
combined_truck_df=combined_truck_df.append(df_donburidc)
combined_truck_df=combined_truck_df.append(df_DorothyMoon1)
combined_truck_df=combined_truck_df.append(df_dougthefooddude)
combined_truck_df=combined_truck_df.append(df_DuckysGrub)
combined_truck_df=combined_truck_df.append(df_elFuegoDc)
combined_truck_df=combined_truck_df.append(df_EndlessPLLC)
combined_truck_df=combined_truck_df.append(df_FarEastTG)
combined_truck_df=combined_truck_df.append(df_Fasikacuisin)
combined_truck_df=combined_truck_df.append(df_FavaPot)
combined_truck_df=combined_truck_df.append(df_FedCityBros)
combined_truck_df=combined_truck_df.append(df_feelincrabby)
combined_truck_df=combined_truck_df.append(df_eatFireNRice)
combined_truck_df=combined_truck_df.append(df_firemanscafe)
combined_truck_df=combined_truck_df.append(df_FoodLady2011)
combined_truck_df=combined_truck_df.append(df_foodforceone1)
combined_truck_df=combined_truck_df.append(df_freshgreenfood)
combined_truck_df=combined_truck_df.append(df_FroZenYoToGo)
combined_truck_df=combined_truck_df.append(df_FCFoodTruck)
combined_truck_df=combined_truck_df.append(df_GeorgesWingTruk)
combined_truck_df=combined_truck_df.append(df_gofishtruckdc)
combined_truck_df=combined_truck_df.append(df_mobilekitchen)
combined_truck_df=combined_truck_df.append(df_GAHDTruck)
combined_truck_df=combined_truck_df.append(df_GREENEGGBURGERS)
combined_truck_df=combined_truck_df.append(df_GridsWaffles)
combined_truck_df=combined_truck_df.append(df_GuaposFoodTruck)
combined_truck_df=combined_truck_df.append(df_habebe19)
combined_truck_df=combined_truck_df.append(df_wherehalal)
combined_truck_df=combined_truck_df.append(df_hardysbbqdivine)
combined_truck_df=combined_truck_df.append(df_healthyfool)
combined_truck_df=combined_truck_df.append(df_henhousedc)
combined_truck_df=combined_truck_df.append(df_HolyCrepesTruck)
combined_truck_df=combined_truck_df.append(df_HotPeopleFood)
combined_truck_df=combined_truck_df.append(df_DCHotWheels)
combined_truck_df=combined_truck_df.append(df_HouseofFalafel)
combined_truck_df=combined_truck_df.append(df_hulagirltruck)
combined_truck_df=combined_truck_df.append(df_hungryheartllc)
combined_truck_df=combined_truck_df.append(df_italiansubs1)
combined_truck_df=combined_truck_df.append(df_JMCCurbside)
combined_truck_df=combined_truck_df.append(df_jerkchickenfest)
combined_truck_df=combined_truck_df.append(df_KababjiTruck)
combined_truck_df=combined_truck_df.append(df_KabobBites)
combined_truck_df=combined_truck_df.append(df_kabob_king)
combined_truck_df=combined_truck_df.append(df_kabobpalace1)
combined_truck_df=combined_truck_df.append(df_KaftaMania)
combined_truck_df=combined_truck_df.append(df_kalaverasdc)
combined_truck_df=combined_truck_df.append(df_KimchiBBQ)
combined_truck_df=combined_truck_df.append(df_dckbbqbox)
combined_truck_df=combined_truck_df.append(df_korengy)
combined_truck_df=combined_truck_df.append(df_kravingkabob)
combined_truck_df=combined_truck_df.append(df_kushimototruck)
combined_truck_df=combined_truck_df.append(df_LATacoTruck)
combined_truck_df=combined_truck_df.append(df_LaTingeriaTruck)
combined_truck_df=combined_truck_df.append(df_latinamericanf1)
combined_truck_df=combined_truck_df.append(df_lemongrasstruck)
combined_truck_df=combined_truck_df.append(df_LilypadontheRun)
combined_truck_df=combined_truck_df.append(df_Limetreetruck)
combined_truck_df=combined_truck_df.append(df_LLuncheonette)
combined_truck_df=combined_truck_df.append(df_LittleItalyDC)
combined_truck_df=combined_truck_df.append(df_chefty99)
combined_truck_df=combined_truck_df.append(df_loslobosburrito)
combined_truck_df=combined_truck_df.append(df_macattackdc)
combined_truck_df=combined_truck_df.append(df_macsdonuts)
combined_truck_df=combined_truck_df.append(df_MakiShopTruck)
combined_truck_df=combined_truck_df.append(df_MamasDonutBites)
combined_truck_df=combined_truck_df.append(df_MayurKababHouse)
combined_truck_df=combined_truck_df.append(df_Med_Delight)
combined_truck_df=combined_truck_df.append(df_Meggrollmania)
combined_truck_df=combined_truck_df.append(df_meskihealthy2go)
combined_truck_df=combined_truck_df.append(df_mesobonwheels)
combined_truck_df=combined_truck_df.append(df_Miamiviceburger)
combined_truck_df=combined_truck_df.append(df_MidniteCC)
combined_truck_df=combined_truck_df.append(df_MightyDogAcai)
combined_truck_df=combined_truck_df.append(df_MisoHoneyTruck)
combined_truck_df=combined_truck_df.append(df_mohmohdumpling)
combined_truck_df=combined_truck_df.append(df_mojaitalatinfla)
combined_truck_df=combined_truck_df.append(df_Mojotruck)
combined_truck_df=combined_truck_df.append(df_Miyagifoodtruck)
combined_truck_df=combined_truck_df.append(df_MuncheezDC)
combined_truck_df=combined_truck_df.append(df_NeatMeatDC)
combined_truck_df=combined_truck_df.append(df_NYDeliTruck)
combined_truck_df=combined_truck_df.append(df_OliviasCupcakes)
combined_truck_df=combined_truck_df.append(df_OohDaTChickeN)
combined_truck_df=combined_truck_df.append(df_orangecowdc)
combined_truck_df=combined_truck_df.append(df_overtherice)
combined_truck_df=combined_truck_df.append(df_parskabob)
combined_truck_df=combined_truck_df.append(df_passl0nfrult)
combined_truck_df=combined_truck_df.append(df_pepefoodtruck)
combined_truck_df=combined_truck_df.append(df_perubrothers)
combined_truck_df=combined_truck_df.append(df_buritosontherun)
combined_truck_df=combined_truck_df.append(df_philliesphamous)
combined_truck_df=combined_truck_df.append(df_phojunkies)
combined_truck_df=combined_truck_df.append(df_PhoBachi)
combined_truck_df=combined_truck_df.append(df_dcphonation)
combined_truck_df=combined_truck_df.append(df_PhoWheels)
combined_truck_df=combined_truck_df.append(df_pleasantpops)
combined_truck_df=combined_truck_df.append(df_PoppedRepublic)
combined_truck_df=combined_truck_df.append(df_dcpuddin)
combined_truck_df=combined_truck_df.append(df_quickwraps)
combined_truck_df=combined_truck_df.append(df_therandyradish)
combined_truck_df=combined_truck_df.append(df_RebasFunnelCake)
combined_truck_df=combined_truck_df.append(df_LobstertruckDC)
combined_truck_df=combined_truck_df.append(df_redbonefc)
combined_truck_df=combined_truck_df.append(df_ReggaeVibesTruk)
combined_truck_df=combined_truck_df.append(df_riochurrascoDC)
combined_truck_df=combined_truck_df.append(df_RitoLoco)
combined_truck_df=combined_truck_df.append(df_RoamingRoti)
combined_truck_df=combined_truck_df.append(df_RocklandsTruck)
combined_truck_df=combined_truck_df.append(df_RockSalt_)
combined_truck_df=combined_truck_df.append(df_Rollinpizzadc)
combined_truck_df=combined_truck_df.append(df_rollinkabob)
combined_truck_df=combined_truck_df.append(df_RollsOnRolls)
combined_truck_df=combined_truck_df.append(df_RovingItalian)
combined_truck_df=combined_truck_df.append(df_asgharboa)
combined_truck_df=combined_truck_df.append(df_saffrontruckdc)
combined_truck_df=combined_truck_df.append(df_SangOnWheels)
combined_truck_df=combined_truck_df.append(df_SaransVegiTruck)
combined_truck_df=combined_truck_df.append(df_SateTruck)
combined_truck_df=combined_truck_df.append(df_Scoops2u)
combined_truck_df=combined_truck_df.append(df_SeoulFoodDC)
combined_truck_df=combined_truck_df.append(df_simple_wheels)
combined_truck_df=combined_truck_df.append(df_simplysoulfood)
combined_truck_df=combined_truck_df.append(df_Sinplicity1)
combined_truck_df=combined_truck_df.append(df_SloppyMamas)
combined_truck_df=combined_truck_df.append(df_BayouGators)
combined_truck_df=combined_truck_df.append(df_smokingkowbbq)
combined_truck_df=combined_truck_df.append(df_solmexicangrill)
combined_truck_df=combined_truck_df.append(df_SouthMeetsEast)
combined_truck_df=combined_truck_df.append(df_SouvlakiStop)
combined_truck_df=combined_truck_df.append(df_SpitfireTruck)
combined_truck_df=combined_truck_df.append(df_steakbites)
combined_truck_df=combined_truck_df.append(df_stellaspopkern)
combined_truck_df=combined_truck_df.append(df_surfsidetruckdc)
combined_truck_df=combined_truck_df.append(df_SUSHIPAO)
combined_truck_df=combined_truck_df.append(df_sweetbitestruck)
combined_truck_df=combined_truck_df.append(df_swizzlerfoods)
combined_truck_df=combined_truck_df.append(df_takorean)
combined_truck_df=combined_truck_df.append(df_tapastruckdc)
combined_truck_df=combined_truck_df.append(df_tashascookies)
combined_truck_df=combined_truck_df.append(df_TasteOfEasternE)
combined_truck_df=combined_truck_df.append(df_TastyFriedDC)
combined_truck_df=combined_truck_df.append(df_tastykabob)
combined_truck_df=combined_truck_df.append(df_TastyToranj)
combined_truck_df=combined_truck_df.append(df_TempoDiPasta)
combined_truck_df=combined_truck_df.append(df_thaimachine)
combined_truck_df=combined_truck_df.append(df_cheesecaketruc)
combined_truck_df=combined_truck_df.append(df_thecornfactory)
combined_truck_df=combined_truck_df.append(df_tfmcupcakery)
combined_truck_df=combined_truck_df.append(df_tiscreamery)
combined_truck_df=combined_truck_df.append(df_thewagonmeal)
combined_truck_df=combined_truck_df.append(df_TinHeavenDC)
combined_truck_df=combined_truck_df.append(df_tokyointhecity)
combined_truck_df=combined_truck_df.append(df_topdogtruck)
combined_truck_df=combined_truck_df.append(df_topstrucks)
combined_truck_df=combined_truck_df.append(df_tortugatruck)
combined_truck_df=combined_truck_df.append(df_BakedTSR)
combined_truck_df=combined_truck_df.append(df_TURKISHKABOBDC)
combined_truck_df=combined_truck_df.append(df_urbanbumpkinbbq)
combined_truck_df=combined_truck_df.append(df_urbanpoutine)
combined_truck_df=combined_truck_df.append(df_VillageCafeINC)
combined_truck_df=combined_truck_df.append(df_wassubdc)
combined_truck_df=combined_truck_df.append(df_WestraysFinest)
combined_truck_df=combined_truck_df.append(df_whatthephodc)
combined_truck_df=combined_truck_df.append(df_wijammincaters)
combined_truck_df=combined_truck_df.append(df_YellaFoodTruck)
combined_truck_df=combined_truck_df.append(df_yellowvendor)
combined_truck_df=combined_truck_df.append(df_YummyYum_Food)
combined_truck_df=combined_truck_df.append(df_yumplingdc)
combined_truck_df=combined_truck_df.append(df_ZestyKabob)
combined_truck_df

,contributors,coordinates,created_at,favorite_count,favorited,geo,id,in_reply_to_status_id,in_reply_to_user_id,is_quote_status,lang,place,retweet_count,retweeted,source,text,truncated,username
0,None,None,2017-02-03 15:57:51,0,False,None,827546651006992384,NaN,NaN,False,en,None,0,False,Facebook,"Happy Aloha Friday L'enfant Plaza!! We are here every Friday for this month so come visit us!! Sorry, our... https://t.co/H3U75kheMs",False,abunaifood
1,None,None,2017-02-02 14:42:40,0,False,None,827165343663669248,NaN,NaN,False,en,None,0,False,Facebook,We have a DOH inspection today...solo again!! Haha 😆 so we are closed today but will be out tomorrow at L'enfant... https://t.co/R6JUL0Kfem,False,abunaifood
2,None,None,2017-02-01 16:48:48,0,False,None,826834699607822336,NaN,NaN,False,tr,None,0,False,Facebook,Chicken Katsu 🐔 #abunaifood #hawaiian #foodtruck #dcfoodtrucks #dcfoodporn #dc #dcfoodie #edibledc #dceatss https://t.co/K6FJXb9B6o,False,abunaifood
3,None,None,2017-02-01 13:49:58,0,False,None,826789692595175425,NaN,NaN,False,en,None,0,False,Twitter for iPhone,We are closed today and tomorrow for caterings and DOH inspection! See you out on Friday at L'enfant Plaza!,False,abunaifood
4,None,None,2017-01-31 21:24:39,1,False,None,826541728110931968,NaN,NaN,False,en,None,0,False,Facebook,Check out our new food truck schedule for February! 😊 #abunaifood #hawaiian #foodtruck #dcfoodtrucks #dcfoodporn... https://t.co/W7lsj5j30v,False,abunaifood
5,None,None,2017-01-30 00:58:29,1,False,None,825870767099371520,NaN,NaN,False,en,None,0,False,Instagram,Heritage Pork bones to broth @messhalldc https://t.co/TI7dNNCDku,False,abunaifood
6,None,None,2017-01-28 18:54:20,1,False,None,825416738896564224,NaN,NaN,False,en,None,0,False,Facebook,Deep clean inside truck day...we work for $150/hour if interested 😂🤣😜😉 #abunaifood #ocdproblems... https://t.co/iGAD7kvix7,False,abunaifood
7,None,None,2017-01-27 20:21:30,0,False,None,825076285798043649,NaN,NaN,False,und,None,0,False,Facebook,https://t.co/ppwYCgRgjb https://t.co/ocyvCoolkD,False,abunaifood
8,None,None,2017-01-27 20:17:49,0,False,None,825075360173793280,NaN,NaN,False,en,None,0,False,Twitter Web Client,Vote Abunai for Best Food Truck in the @wcp #bestofdc 2017 Poll: https://t.co/F7D73zoj4l,False,abunaifood
9,None,None,2017-01-27 16:56:04,0,False,None,825024588371394560,NaN,NaN,False,en,None,0,False,Instagram,Happy Aloha Friday Chinatown! Come visit us!! We got stuck in this protest! 😂 If you can't make… https://t.co/qdPl3fQWIU,False,abunaifood


In [54]:
#Write the combined df to csv for further analysis
combined_truck_df_csv_path = "/Users/nicolekelly/Documents/ft_aggregate/" + str(time.time()) + '_' + 'combined_truck_df' +'.csv'
combined_truck_df.to_csv(path_or_buf=combined_truck_df_csv_path, header=['contributors', 'coordinates', 'created_at', 'favorite_count', 'favorited', 'geo', 'id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'is_quote_status', 'lang', 'place', 'retweet_count', 'retweeted', 'source', 'text', 'truncated', 'username'], index=True, sep=',')

In [ ]:
#Next steps:
    #1)Use pandas to find food trucks where the most recent tweet occurred before 2016 (as a starting point)
    # to remove from analysis
    #2)Reorder columns in df for more logical order
    #3)Use regex to extract additional info (like RT=retweet, hashtags, etc)
    #4)Figure out how to get information from nested dictionaries, like retweets
    #5)Get more complete dataset for most prolific tweeters (using max and since ids in API call)
    #6)Start storing dataset in SQL/Postgres to add new info to old more easily
    #7)Analyze!

In [15]:
#doesn't work yet, something with the pd.DataFrame line syntax
trucks=['captaincookiedc']
for food_truck_twitter_handle in trucks:
    food_truck_twitter_handle_timeline=api.user_timeline('{}'.format(food_truck_twitter_handle))
    food_truck_twitter_handle_tweets=[]
    for tweet in food_truck_twitter_handle_timeline:
        tweet_parts = {
            "id":tweet.id,
            "created_at":tweet.created_at,
            "text":tweet.text
        }
        food_truck_twitter_handle_tweets.append(tweet_parts)
    food_truck_twitter_handle_df_2 = pd.DataFrame(food_truck_twitter_handle_tweets)
    #return food_truck_twitter_handle_df
#captaincookiedc_df_2


NameError: name 'captaincookiedc_timeline' is not defined

In [3]:
#Pull in most recent 20 tweets
ArepaZone_timeline=api.user_timeline("ArepaZone")
#Set a number to loop through
i=-1
#Create arrays for each feature of the timeline json object to extract
ArepaZone_id_array=numpy.empty(len(ArepaZone_timeline),dtype=object)
ArepaZone_date_array=numpy.empty(len(ArepaZone_timeline),dtype=object)
ArepaZone_tweet_array=numpy.empty(len(ArepaZone_timeline),dtype=object)
#Loop through the tweet objects and add the feature contents to the relevant arrays
for tweet in ArepaZone_timeline:
    i=i+1
    ArepaZone_id_array[i]=tweet.id
    ArepaZone_date_array[i]=tweet.created_at
    ArepaZone_tweet_array[i]=tweet.text


#Create the pandas dataframe using the ID array to merge the other features into
ArepaZone_merged_arrays_df = pd.DataFrame(data=ArepaZone_id_array, columns=['id']) 


#Create dataframes for the other features so they are able to be merged
ArepaZone_tweet_array_df = pd.DataFrame(data=ArepaZone_tweet_array, columns=['tweet']) 
ArepaZone_date_array_df = pd.DataFrame(data=ArepaZone_date_array, columns=['date']) 


#Merge the other feature dataframes into the main dataframe
ArepaZone_merged_arrays_df=ArepaZone_merged_arrays_df.merge(ArepaZone_tweet_array_df,left_index=True, right_index=True)
ArepaZone_merged_arrays_df=ArepaZone_merged_arrays_df.merge(ArepaZone_date_array_df,left_index=True, right_index=True)


#Output the dataframe to a text file to save it (will eventually want to do this in SQL)


In [4]:
#These are functions of all of these commands above
def tweets_to_dataframe(food_truck_twitter_handle, food_truck_twitter_handle_string):
    food_truck_twitter_handle_timeline=""
    food_truck_twitter_handle_timeline=api.user_timeline('{}'.format(food_truck_twitter_handle_string))
    #Set a number to loop through
    i=-1
    #Create arrays for each feature of the timeline json object to extract
    food_truck_twitter_handle_id_array=numpy.empty(len('{}_timeline'.format(food_truck_twitter_handle_string)),dtype=object)
    food_truck_twitter_handle_date_array=numpy.empty(len('{}_timeline'.format(food_truck_twitter_handle_string)),dtype=object)
    food_truck_twitter_handle_tweet_array=numpy.empty(len('{}_timeline'.format(food_truck_twitter_handle_string)),dtype=object)
    #Loop through the tweet objects and add the feature contents to the relevant arrays
    for tweet in food_truck_twitter_handle_timeline:
        i=i+1
        food_truck_twitter_handle_id_array[i]=tweet.id
        food_truck_twitter_handle_date_array[i]=tweet.created_at
        food_truck_twitter_handle_tweet_array[i]=tweet.text
    #Create the pandas dataframe using the ID array to merge the other features into
    food_truck_twitter_handle_merged_arrays_df = pd.DataFrame(data=food_truck_twitter_handle_id_array, columns=['id']) 
    #food_truck_twitter_handle_merged_arrays_df = pd.DataFrame(data=eval('{}_id_array'.format(food_truck_twitter_handle_string)), columns=['id']) 
    #Create dataframes for the other features so they are able to be merged
    food_truck_twitter_handle_tweet_array_df = pd.DataFrame(data=food_truck_twitter_handle_tweet_array, columns=['tweet']) 
    food_truck_twitter_handle_date_array_df = pd.DataFrame(data=food_truck_twitter_handle_date_array, columns=['date']) 
    #Merge the other feature dataframes into the main dataframe
    food_truck_twitter_handle_merged_arrays_df=food_truck_twitter_handle_merged_arrays_df.merge(food_truck_twitter_handle_tweet_array_df,left_index=True, right_index=True)
    food_truck_twitter_handle_merged_arrays_df=food_truck_twitter_handle_merged_arrays_df.merge(food_truck_twitter_handle_date_array_df,left_index=True, right_index=True)
    log_tweets = "/Users/nicolekelly/Documents/ft_aggregate/" + str(time.time()) + '_' + '{}'.format(food_truck_twitter_handle_string)+'.csv'
    food_truck_twitter_handle_merged_arrays_df.to_csv(path_or_buf=log_tweets, header=['ID','Tweet','Date'], index=True, sep=',')
    return food_truck_twitter_handle_merged_arrays_df


In [12]:
healthyfool_merged_arrays_df=tweets_to_dataframe('healthyfool', 'healthyfool')
#abunai_merged_arrays_df=tweets_to_dataframe('abunaifood', 'abunaifood')
ballornothing_merged_arrays_df=tweets_to_dataframe('theballtruck', 'theballtruck')
arepazone_merged_arrays_df=tweets_to_dataframe('ArepaZone','ArepaZone')


IndexError: index 18 is out of bounds for axis 0 with size 18

In [5]:
ArepaZone_merged_arrays_df

,id,tweet,date
0,817029652392931330,Farragut Square today 🙌 https://t.co/zoxKGV2XXR,2017-01-05 15:27:03
1,816749041182801921,Looking for dinner? Our Union Market stall is ...,2017-01-04 20:52:00
2,816566606050500608,"@magneticdynamo not this week, but we are next...",2017-01-04 08:47:04
3,816430441242750976,"Tomorrow's food truck spot is 19th and L, NW! ...",2017-01-03 23:46:00
4,816386654659366913,Thanks for joining us for lunch! For those who...,2017-01-03 20:52:00
5,816305643900596224,Getting ready to open our window at Franklin S...,2017-01-03 15:30:06
6,816262061013475328,Happy New Year! We're super excited for this y...,2017-01-03 12:36:55
7,814855551054204928,Catch us one last time this year! We're parked...,2016-12-30 15:27:57
8,814541106566365184,@Debbliss99 solo por encargo! Llámanos al 7032...,2016-12-29 18:38:27
9,814476576838586369,We are headed to Farragut Square one last time...,2016-12-29 14:22:02


In [ ]:
ArepaZone_timeline

In [18]:
#Pull in most recent 20 tweets
ArepaZone_timeline=api.user_timeline("ArepaZone")
#ArepaZone_timeline_json=json.dumps([status._json for status in ArepaZone_timeline])
#ArepaZone_timeline_json
#with open('ArepaZone_json_output.json', 'w') as outfile:
#    json.dump(ArepaZone_timeline_json, outfile)
#ArepaZone_json_df = pd.read_json('ArepaZone_json_output.json')
#ArepaZone_json_df
#print (dir(ArepaZone_timeline))
#print(ArepaZone_timeline.ids)
ArepaZone_timeline_json=ArepaZone_timeline[0]
ArepaZone_timeline_json_str = json.dumps(ArepaZone_timeline._json)




AttributeError: 'ResultSet' object has no attribute '_json'

In [ ]:
#Set a number to loop through
iterator=-1
#Create arrays for each feature of the timeline json object to extract
ArepaZone_id_array=numpy.empty(len(ArepaZone_timeline),dtype=object)
ArepaZone_date_array=numpy.empty(len(ArepaZone_timeline),dtype=object)
ArepaZone_tweet_array=numpy.empty(len(ArepaZone_timeline),dtype=object)
#Loop through the tweet objects and add the feature contents to the relevant arrays
for tweet in ArepaZone_timeline:
    iterator=iterator+1
    ArepaZone_id_array[iterator]=tweet.id
    ArepaZone_date_array[iterator]=tweet.created_at
    ArepaZone_tweet_array[iterator]=tweet.text

In [ ]:
#Create the pandas dataframe using the ID array to merge the other features into
ArepaZone_merged_arrays_df = pd.DataFrame(data=ArepaZone_id_array, columns=['id']) 

In [ ]:
#Create dataframes for the other features so they are able to be merged
ArepaZone_tweet_array_df = pd.DataFrame(data=ArepaZone_tweet_array, columns=['tweet']) 
ArepaZone_date_array_df = pd.DataFrame(data=ArepaZone_date_array, columns=['date']) 

In [ ]:
#Merge the other feature dataframes into the main dataframe
ArepaZone_merged_arrays_df=ArepaZone_merged_arrays_df.merge(ArepaZone_tweet_array_df,left_index=True, right_index=True)
ArepaZone_merged_arrays_df=ArepaZone_merged_arrays_df.merge(ArepaZone_date_array_df,left_index=True, right_index=True)

In [ ]:
#Output the dataframe to a text file to save it (will eventually want to do this in SQL)


In [ ]:
ArepaZone_merged_arrays_df

In [ ]:
#These are functions of all of these commands above
def tweets_to_dataframe(food_truck_twitter_handle, food_truck_twitter_handle_string):
    food_truck_twitter_handle_timeline=""
    food_truck_twitter_handle_timeline=api.user_timeline('{}'.format(food_truck_twitter_handle_string))
    #Set a number to loop through
    #Create lists
    food_truck_twitter_handle_id_array=[0]
    food_truck_twitter_handle_date_array=[0]
    food_truck_twitter_handle_tweet_array=[""]
    #Create arrays for each feature of the timeline json object to extract
    #food_truck_twitter_handle_id_array=numpy.empty(len('{}_timeline'.format(food_truck_twitter_handle_string)),dtype=object)
    #food_truck_twitter_handle_date_array=numpy.empty(len('{}_timeline'.format(food_truck_twitter_handle_string)),dtype=object)
    #food_truck_twitter_handle_tweet_array=numpy.empty(len('{}_timeline'.format(food_truck_twitter_handle_string)),dtype=object)
    #Loop through the tweet objects and add the feature contents to the relevant arrays
    #iterator=len('{}_timeline'.format(food_truck_twitter_handle_string))-len('{}_timeline'.format(food_truck_twitter_handle_string))
    for tweet in food_truck_twitter_handle_timeline:
        food_truck_twitter_handle_id_array=food_truck_twitter_handle_id_array.append(getattr(tweet, 'id', None))
        #food_truck_twitter_handle_date_array[iterator]=getattr(tweet, 'created_at', None)
        #food_truck_twitter_handle_tweet_array[iterator]=getattr(tweet, 'text', None)
        #iterator=iterator+1
    #Create the pandas dataframe using the ID array to merge the other features into
    food_truck_twitter_handle_merged_arrays_df = pd.DataFrame(data=food_truck_twitter_handle_id_array, columns=['id']) 
    #food_truck_twitter_handle_merged_arrays_df = pd.DataFrame(data=eval('{}_id_array'.format(food_truck_twitter_handle_string)), columns=['id']) 
    #Create dataframes for the other features so they are able to be merged
    #food_truck_twitter_handle_tweet_array_df = pd.DataFrame(data=food_truck_twitter_handle_tweet_array, columns=['tweet']) 
    #food_truck_twitter_handle_date_array_df = pd.DataFrame(data=food_truck_twitter_handle_date_array, columns=['date']) 
    #Merge the other feature dataframes into the main dataframe
    #food_truck_twitter_handle_merged_arrays_df=food_truck_twitter_handle_merged_arrays_df.merge(food_truck_twitter_handle_tweet_array_df,left_index=True, right_index=True)
    #food_truck_twitter_handle_merged_arrays_df=food_truck_twitter_handle_merged_arrays_df.merge(food_truck_twitter_handle_date_array_df,left_index=True, right_index=True)
    log_tweets = "/Users/nicolekelly/Documents/ft_aggregate/" + str(time.time()) + '_' + '{}'.format(food_truck_twitter_handle_string)+'.csv'
    food_truck_twitter_handle_merged_arrays_df.to_csv(path_or_buf=log_tweets, header=['ID'], index=True, sep=',')
    #food_truck_twitter_handle_merged_arrays_df.to_csv(path_or_buf=log_tweets, header=['ID','Tweet','Date'], index=True, sep=',')
    return food_truck_twitter_handle_merged_arrays_df

In [ ]:
#These are functions of all of these commands above
#trucks=['ArepaZone','abunaifood', 'healthyfool', 'theballtruck']
trucks=['captaincookiedc']
for food_truck_twitter_handle in trucks:
    food_truck_twitter_handle_timeline=api.user_timeline('{}'.format(food_truck_twitter_handle))
    #Create arrays for each feature of the timeline json object to extract
    food_truck_twitter_handle_id_array=numpy.empty(len('{}_timeline'.format(food_truck_twitter_handle)),dtype=object)
    food_truck_twitter_handle_date_array=numpy.empty(len('{}_timeline'.format(food_truck_twitter_handle)),dtype=object)
    food_truck_twitter_handle_tweet_array=numpy.empty(len('{}_timeline'.format(food_truck_twitter_handle)),dtype=object)
    #Loop through the tweet objects and add the feature contents to the relevant arrays
    iterator=len('{}_timeline'.format(food_truck_twitter_handle))-len('{}_timeline'.format(food_truck_twitter_handle))
    for tweet in food_truck_twitter_handle_timeline:
        if getattr(tweet, 'id', None) != None 
        food_truck_twitter_handle_id_array[iterator]=getattr(tweet, 'id', None)
        food_truck_twitter_handle_date_array[iterator]=getattr(tweet, 'created_at', None)
        food_truck_twitter_handle_tweet_array[iterator]=getattr(tweet, 'text', None)
        iterator=iterator+1
    #Create the pandas dataframe using the ID array to merge the other features into
    food_truck_twitter_handle_merged_arrays_df = pd.DataFrame(data=food_truck_twitter_handle_id_array, columns=['id']) 
    #food_truck_twitter_handle_merged_arrays_df = pd.DataFrame(data=eval('{}_id_array'.format(food_truck_twitter_handle_string)), columns=['id']) 
    #Create dataframes for the other features so they are able to be merged
    food_truck_twitter_handle_tweet_array_df = pd.DataFrame(data=food_truck_twitter_handle_tweet_array, columns=['tweet']) 
    food_truck_twitter_handle_date_array_df = pd.DataFrame(data=food_truck_twitter_handle_date_array, columns=['date']) 
    #Merge the other feature dataframes into the main dataframe
    food_truck_twitter_handle_merged_arrays_df=food_truck_twitter_handle_merged_arrays_df.merge(food_truck_twitter_handle_tweet_array_df,left_index=True, right_index=True)
    food_truck_twitter_handle_merged_arrays_df=food_truck_twitter_handle_merged_arrays_df.merge(food_truck_twitter_handle_date_array_df,left_index=True, right_index=True)
    log_tweets = "/Users/nicolekelly/Documents/ft_aggregate/" + str(time.time()) + '_' + '{}'.format(food_truck_twitter_handle)+'.csv'
    #food_truck_twitter_handle_merged_arrays_df.to_csv(path_or_buf=log_tweets, header=['ID'], index=True, sep=',')
    food_truck_twitter_handle_merged_arrays_df.to_csv(path_or_buf=log_tweets, header=['ID','Tweet','Date'], index=True, sep=',')
    #return food_truck_twitter_handle_merged_arrays_df

In [ ]:
food_truck_twitter_handle_id_array

In [ ]:
healthyfool_merged_arrays_df=tweets_to_dataframe('healthyfool', 'healthyfool')
abunai_merged_arrays_df=tweets_to_dataframe('abunaifood', 'abunaifood')
ballornothing_merged_arrays_df=tweets_to_dataframe('theballtruck', 'theballtruck')
arepazone_merged_arrays_df=tweets_to_dataframe('ArepaZone','ArepaZone')

In [ ]:
abunai_merged_arrays_df=tweets_to_dataframe('abunaifood', 'abunaifood')

In [ ]:
healthyfool_merged_arrays_df

In [19]:
#test code, no longer in use
ArepaZone_timeline=api.user_timeline("ArepaZone")
#type(ArepaZone_timeline)
json_str=json.dumps([status._json for status in ArepaZone_timeline])
parsed_json=json.loads(json_str)

In [ ]:
json_str

In [ ]:
type(parsed_json)
print(dir(parsed_json))
#print(parsed_json['text'])
parsed_json.items()

In [90]:
new_array=numpy.empty([20,3], dtype=str)
for tweet in parsed_json:
    #print(dir(tweet))
    #print(tweet[('text','id','created_at')])
    counter=len(parsed_json)-len(parsed_json)
    arepa_zone_array[counter]=map(tweet.get, ['text','id','created_at'])
    counter=counter+1
    


TypeError: float() argument must be a string or a number, not 'map'

In [66]:

arepa_zone_list
len(parsed_json)

20

In [ ]:
#json_str

In [ ]:
#User Timeline function brings in all sorts of information about the account/tweet
#I only care about the text string
#Regular expression that captures section that starts with "text:"
#I might also want dates soon too
#regex_screenname_id = r"\"contributors_enabled\"\:\s\w+\,\s\"id\"\:\s(\w+)"
#regex_screenname = r"\"screen_name\"\:\s\"(.*?)\""
#regex_tweet_id = r"\"id\"\:\s(\w+)\,\s\"text"
#regex_date = r"\"created_at\"\:\s\"(.*?)\""
#regex_text = r"\"text\"\:\s\"(.*?)\""

In [ ]:
#Function that pulls in the most recent 20 tweets (I'm not sure if I can grab more than 20 right now)
#def getTimeline(foodtruckname, foodtruckname_str):
#    foodtruckname_timeline=""
#    foodtruckname_timeline=api.user_timeline('{}'.format(foodtruckname_str))
#    foodtruckname_json_str=json.dumps([status._json for status in foodtruckname_timeline])
#    foodtruckname_screenname_id = re.findall(regex_screenname_id, foodtruckname_json_str, flags=0)
#    foodtruckname_screenname = re.findall(regex_screenname, foodtruckname_json_str, flags=0)
#    foodtruckname_tweet_id = re.findall(regex_tweet_id, foodtruckname_json_str, flags=0)
#    foodtruckname_date = re.findall(regex_date, foodtruckname_json_str, flags=0)
#    foodtruckname_text = re.findall(regex_text, foodtruckname_json_str, flags=0)
#    return {'foodtruckname_screenname_id':foodtruckname_screenname_id, 'foodtruckname_screenname': foodtruckname_screenname,
#            'foodtruckname_tweet_id':foodtruckname_tweet_id, 'foodtruckname_date':foodtruckname_date, 
#            'foodtruckname_text':foodtruckname_text}

In [ ]:
#ArepaZone_tweets=getTimeline('ArepaZone', 'ArepaZone')
#ArepaZone_tweets

In [ ]:
#ArepaZone_df=pd.DataFrame.from_dict(ArepaZone_tweets,orient='columns')

In [ ]:
#tweet_data = []
    # You will use this line in production instead of this
    # current_working_dir = os.path.dirname(os.path.realpath(__file__))
#current_working_dir = "./"
#log_tweets = current_working_dir  + str(time.time()) + '_fttweets.txt'
#with open(log_tweets, 'w') as outfile:
#    for tweet in ArepaZone_timeline:
#        tweet_data.append(json.loads(json.dumps(tweet._json)))
#        outfile.write(json.dumps(tweet._json))
#        outfile.write("\n")

In [ ]:
# Create the dataframe we will use
#tweets = pd.DataFrame()
# We want to know when a tweet was sent
#tweets['created_at'] = map(lambda tweet: time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(tweet['created_at'],'%a %b %d %H:%M:%S +0000 %Y')), tweet_data)
# Who is the tweet owner
#tweets['user'] = map(lambda tweet: tweet['user']['screen_name'], tweet_data)
# How many follower this user has
#tweets['user_followers_count'] = map(lambda tweet: tweet['user']['followers_count'], tweet_data)
# What is the tweet's content
#tweets['text'] = map(lambda tweet: tweet['text'].encode('utf-8'), tweet_data)
# If available what is the language the tweet is written in
#tweets['lang'] = map(lambda tweet: tweet['lang'], tweet_data)
# If available, where was the tweet sent from ?
#tweets['Location'] = map(lambda tweet: tweet['place']['country'] if tweet['place'] != None else None, tweet_data)
# How many times this tweet was retweeted and favorited
#tweets['retweet_count'] = map(lambda tweet: tweet['retweet_count'], tweet_data)
#tweets['favorite_count'] = map(lambda tweet: tweet['favorite_count'], tweet_data)


In [ ]:
#print(tweets.head())

In [ ]:
#tweets_raw_data = []
i=-1
#index=range(len(ArepaZone_timeline))
#ArepaZone_id=pd.DataFrame(index=index)
ArepaZone_id=numpy.empty(len(ArepaZone_timeline),dtype=object)
ArepaZone_date=numpy.empty(len(ArepaZone_timeline),dtype=object)
ArepaZone_tweets=numpy.empty(len(ArepaZone_timeline),dtype=object)
for tweet in ArepaZone_timeline:
    i=i+1
    ArepaZone_tweets[i]=tweet.text
    ArepaZone_id[i]=tweet.id
    ArepaZone_date[i]=tweet.created_at
    #print(tweet.text)
    #tweets_raw_data.append(tweet)
    # Create the dataframe we will use
#tweets = pd.DataFrame()

In [ ]:
len(ArepaZone_timeline)

In [ ]:
#ArepaZone_timeline
ArepaZone_id

In [ ]:
ArepaZone_id

In [ ]:
# Create the dataframe we will use
ArepaZone_tweetDF = pd.DataFrame(data=ArepaZone_id, columns=['id']) 

In [ ]:
ArepaZone_tweetsDF = pd.DataFrame(data=ArepaZone_tweets, columns=['tweet']) 
ArepaZone_datesDF = pd.DataFrame(data=ArepaZone_date, columns=['date']) 

In [ ]:
ArepaZone_tweetDF=ArepaZone_tweetDF.merge(ArepaZone_tweetsDF,left_index=True, right_index=True)
ArepaZone_tweetDF=ArepaZone_tweetDF.merge(ArepaZone_datesDF, left_index=True, right_index=True)

In [ ]:
ArepaZone_tweetDF

In [ ]:
#Pull in most recent 20 tweets
abunai_timeline=api.user_timeline("abunaifood")
abunai_timeline

In [ ]:
#Set a number to loop through
i=-1
#Create arrays for each feature of the timeline json object to extract
abunai_id_array=numpy.empty(len(abunai_timeline),dtype=object)
abunai_date_array=numpy.empty(len(abunai_timeline),dtype=object)
abunai_tweet_array=numpy.empty(len(abunai_timeline),dtype=object)
#Loop through the tweet objects and add the feature contents to the relevant arrays
for tweet in abunai_timeline:
    i=i+1
    abunai_id_array[i]=getattr(tweet, 'id', None)
    abunai_date_array[i]=tweet.created_at
    abunai_tweet_array[i]=tweet.text

In [ ]:
#Create the pandas dataframe using the ID array to merge the other features into
abunai_merged_arrays_df = pd.DataFrame(data=abunai_id_array, columns=['id']) 

In [ ]:
#Create dataframes for the other features so they are able to be merged
abunai_tweet_array_df = pd.DataFrame(data=abunai_tweet_array, columns=['tweet']) 
abunai_date_array_df = pd.DataFrame(data=abunai_date_array, columns=['date']) 

In [ ]:
#Merge the other feature dataframes into the main dataframe
abunai_merged_arrays_df=abunai_merged_arrays_df.merge(abunai_tweet_array_df,left_index=True, right_index=True)
abunai_merged_arrays_df=abunai_merged_arrays_df.merge(abunai_date_array_df,left_index=True, right_index=True)

In [ ]:
abunai_merged_arrays_df

In [17]:
#Trying David's method of creating a dictionary out of the json
#Pull in most recent 20 tweets
ArepaZone_timeline=api.user_timeline("ArepaZone")
#type(ArepaZone_timeline)
#Create dictionary
ArepaZone_dictionary={}
#Loop through the tweet objects and add the feature contents to the relevant arrays
for tweet in ArepaZone_timeline:
    ArepaZone_dictionary[tweet]={"id":tweet.id, "text":tweet.text, "date":tweet.created_at}


TypeError: unhashable type: 'Status'

In [15]:
#trying the above but trying to convert the unhashable status object into a json object
json_str=json.dumps([status._json for status in ArepaZone_timeline])
#parsed_json=json.loads(json_str)
#type(parsed_json)
type(json_str)

str

In [21]:
#Trying David's method of creating a dictionary out of the json
#Pull in most recent 20 tweets
ArepaZone_timeline=api.user_timeline("ArepaZone")
#Create dictionary
ArepaZone_dictionary={}
#Loop through the tweet objects and add the feature contents to the relevant arrays
for tweet in parsed_json:
    ArepaZone_dictionary[tweet]={"id":tweet.id, "text":tweet.text, "date":tweet.created_at}

AttributeError: 'dict' object has no attribute 'text'

In [ ]:
#api.user_timeline() brings in an object called tweepy.models.ResultSet
#This can be converted to a str object via json_str=json.dumps([status._json for status in ArepaZone_timeline])
#Or then a list object via json.loads(json_str)
#what can I do with the string object or the list object
dir(json_str)

In [ ]:
parsed_json

In [6]:
#second method from David
#Pull in most recent 20 tweets
ArepaZone_timeline=api.user_timeline("ArepaZone")
arepa_tweets=[]
for tweet in ArepaZone_timeline:
    tweet_parts = {
        "id":tweet.id,
        "created_at":tweet.created_at,
        "text":tweet.text
    }
    arepa_tweets.append(tweet_parts)

ArepaZone_df = pd.DataFrame(arepa_tweets)
ArepaZone_df

In [ ]:
ArepaZone_df

# List of all food trucks
## from foodtruckfiesta.com

    Abunai (abunaifood)
    Amorini Panini (AmoriniPanini)
    Angry Burger (AngryBurgerNOW)
    Arepa Crew (arepacrew)
    Arepa Zone (ArepaZone)
    Astro Doughnuts & Fried Chicken (AstroDoughnuts)
    ATIP (Partytrucks)
    AZN Eats (AZNeats)
    A’ Lo Cubano (alocubanotruck)
    Baba's Big Bite (BabasBigBite)
    Bada Bing (badabingdc)
    Balkanik Taste (balkaniktaste)
    Ball or Nothing (theballtruck)
    Banh Mi Annie (BanhMiAnnie)
    Basil Thyme (BasilThymeDC)
    BBQ Bus (bbqbusdc)
    Beach Fries (gotbeachfries)
    Beirut Delights (BeirutDelights)
    Bella Vita Italian Wheels (bellavitawheels)
    Best Mexican Burritos and Tortas (bestburritos1)
    BiBi Ja (okbibija)
    Big Cheese (bigcheesetruck)
    Bite 2 Go (BITE2GO)
    BONMi (eatBONMi)
    Borinquen Lunch Box (Borinquenlunchb)
    Brandon's Little Truck (BLTfoodtruck)
    Bratwurst King (bratwurstking)
    Brooklyn Sandwich Co (bklynsandwichco)
    Brown Bag (eatbrownbag)
    Bubble Tea Licious (bubbleteaparty)
    BurGorilla (burgorilla)
    CA Slider Company (CAslidercompany)
    Cajunators (cajunators)
    Capital Chicken & Waffles (CapitalCW)
    CapMac (capmacdc)
    Captain Cookie and the Milk Man (CaptainCookieDC)
    Caribbean Cafe (Caribbean2Go)
    Carmen's Italian Ice (CarmensCarts)
    Carnivore BBQ (carnbbq)
    Carolina Q (EatCarolinaQ)
    Cathy's Bistro (1styellowvendor)
    Chatpat (ChatpatTruck)
    Cheesequake (cheesequakedc)
    Chef Alex (chefalex2013)
    Chef On Wheels (ChefonWheels11)
    Chef Seb (chefsebonwheels)
    Chez Adilmos (ChezAdilmos)
    Chick-fil-A (chickfilamobile)
    Chitti Chitti Grill (ChittiGrill)
    Chix N Stix (ChixNStixDC)
    Coles Palette (coles_palette)
    Corned Beef King (cornedbeefking)
    Crab Cab (CrabCab)
    Crave It (CRAVEIT2)
    Creme de la Cupcake (cremedelacakedc)
    Crêpe Love (crepelovetruck)
    Crepes Parfait (CrepesParfait)
    Cupcake Joy (JoyCupcakes)
    Curbside Crabcakes (CurbSideCrab)
    Curbside Cupcakes (curbsidecupcake)
    Curley's Q (CurleysQ)
    Dangerous Pies (ThePieTruckDC)
    DC Ballers (DcBallers)
    DC Doner (dc_doner)
    DC Empanadas (DCEmpanadas)
    DC Greek Food (dc_greekfood)
    DC Pollo (dcpollos)
    DC Quesadilla & Wraps (DCQuesadilla)
    DC Slices (dcslices)
    DC Sliders (DCSLIDERSTRUCK)
    DC Taco Truck (DCTacoTruck)
    Dirty South Deli (dirtysouthdeli)
    Dolci Gelati (DolciGelatiTruk)
    Donburi (donburidc)
    Dorothy Moon’s Gourmet Burgers (DorothyMoon1)
    Doug The Food Dude (dougthefooddude)
    Ducky's Grub (DuckysGrub)
    El Fuego (elFuegoDc)
    Endless Pastabilities (EndlessPLLC)
    Far East Taco Grille (FarEastTG)
    Fasika Ethiopian Cuisine (Fasikacuisin)
    Fava Pot (FavaPot)
    Federal City Bros (FedCityBros)
    Feelin' Crabby (feelincrabby)
    Fire & Rice (eatFireNRice)
    Firemans Cafe (firemanscafe)
    Food For The Soul (FoodLady2011)
    Food Force One (foodforceone1)
    Fresh Green (freshgreenfood)
    FroZenYo To Go (FroZenYoToGo)
    Fusion Confusion (FCFoodTruck)
    George's Buffalo Wings (GeorgesWingTruk)
    Go Fish! (gofishtruckdc)
    Goode's Mobile Kitchen (mobilekitchen)
    Greatest American Hot Dogs (GAHDTruck)
    Green Eggs and Burgers (GREENEGGBURGERS)
    Grids Waffles (GridsWaffles)
    Guapo's (GuaposFoodTruck)
    Habebe (habebe19)
    Halal Grill (wherehalal)
    Hardy's BBQ (hardysbbqdivine)
    Healthy Fool (healthyfool)
    Henhouse (henhousedc)
    Holy Crepes (HolyCrepesTruck)
    Hot People Food (HotPeopleFood)
    Hot Wheels (DCHotWheels)
    House Of Falafel (HouseofFalafel)
    Hula Girl Truck (hulagirltruck)
    Hungry Heart (hungryheartllc)
    Italian Subs (italiansubs1)
    Jamaican Mi Crazy (JMCCurbside)
    Jerk Chicken Festival (jerkchickenfest)
    Kababji Food Truck (KababjiTruck)
    Kabob Bites (KabobBites)
    Kabob King (kabob_king)
    Kabob Palace (kabobpalace1)
    Kafta Mania (KaftaMania)
    Kalaveras (kalaverasdc)
    Kimchi BBQ Taco (KimchiBBQ)
    Korean BBQ Taco Box (dckbbqbox)
    Korengy (korengy)
    Kraving Kabob (kravingkabob)
    Kushi-Moto (kushimototruck)
    LA Taco Truck (LATacoTruck)
    La Tingeria (LaTingeriaTruck)
    Latin & American Flavors (latinamericanf1)
    Lemongrass Truck (lemongrasstruck)
    Lilypad On The Run (LilypadontheRun)
    Lime Tree (Limetreetruck)
    Linda's Luncheonette (LLuncheonette)
    Little Italy On Wheels (LittleItalyDC)
    Little Piece of Heaven (chefty99)
    Los Lobos Burritos (loslobosburrito)
    Mac Attack (macattackdc)
    Mac's Donuts (macsdonuts)
    Maki Shop (MakiShopTruck)
    Mama's Donut Bites (MamasDonutBites)
    Mayur Kabob House (MayurKababHouse)
    Mediterranean Delight (Med_Delight)
    Meggrolls (Meggrollmania)
    Meski Healthy 2 Go (meskihealthy2go)
    Mesob On Wheels (mesobonwheels)
    Miami Vice Burgers (Miamiviceburger)
    Midnite Confections Cupcakery (MidniteCC)
    Mighty Dog And Acai (MightyDogAcai)
    Miso Honey (MisoHoneyTruck)
    Moh Moh Dumpling (mohmohdumpling)
    Mojaita Latin Flavor (mojaitalatinfla)
    MojoTruck (Mojotruck)
    Mr Miyagi’s Teriyaki (Miyagifoodtruck)
    Muncheez (MuncheezDC)
    Naan Stop DC (NaanStopDC)
    NeatMeat (NeatMeatDC)
    New York Deli (NYDeliTruck)
    Olivia's Cupcakes (OliviasCupcakes)
    OoH DaT ChickeN (OohDaTChickeN)
    Orange Cow (orangecowdc)
    Over the Rice (overtherice)
    Pars Kabob (parskabob)
    Passion Fruit (passl0nfrult)
    Pepe (pepefoodtruck)
    Peruvian Brothers (perubrothers)
    Philadelphia Cheesesteak Express (buritosontherun)
    Phillies Phamous Cheesesteaks (philliesphamous)
    Pho Junkies (phojunkies)
    Pho-Bachi (PhoBachiInc)
    Phonation (dcphonation)
    PhoWheels (PhoWheels)
    Pleasant Pops (pleasantpops)
    Popped! Republic (PoppedRepublic)
    Puddin' (dcpuddin)
    Quick Wraps (quickwraps)
    Randy Radish (therandyradish)
    Reba's Funnel Cake (RebasFunnelCake)
    Red Hook Lobster Pound (LobstertruckDC)
    Redbone (redbonefc)
    Reggae Vibes (ReggaeVibesTruk)
    Rio Churrasco (riochurrascoDC)
    Rito Loco (RitoLoco)
    Roaming Rotisserie (RoamingRoti)
    Rocklands BBQ (RocklandsTruck)
    RockSalt (RockSalt_)
    Rollin Pizza (Rollinpizzadc)
    Rollin' Kabob (rollinkabob)
    Rolls On Rolls (RollsOnRolls)
    Roving Italian (RovingItalian)
    Royal Chicken & Gyro (asgharboa)
    Saffron Food Lovers (saffrontruckdc)
    Sang On Wheels (SangOnWheels)
    Saran's Vegetarian (SaransVegiTruck)
    Sate (SateTruck)
    Scoops2u (Scoops2u)
    Señor Taco (senortaco7)
    Seoul Food (SeoulFoodDC)
    Sidewalk Sweets (sidewalksweets)
    Simple on Wheels (simple_wheels)
    Simply Delightful (simplysoulfood)
    Sinplicity Ice Cream (Sinplicity1)
    Sloppy Mama's (SloppyMamas)
    Smokin' On The Bayou (BayouGators)
    Smoking Kow BBQ (smokingkowbbq)
    SOL Mexican Grill (solmexicangrill)
    South Meets East (SouthMeetsEast)
    Souvlaki Stop (SouvlakiStop)
    Spitfire (SpitfireTruck)
    Steak Bites (steakbites)
    Stella’s Popkern (stellaspopkern)
    Street Cream (StreetCream34)
    SUNdeVICH (SUNdeVICHtruck)
    Surfside (surfsidetruckdc)
    SUSHIPAO (SUSHIPAO)
    Sweetbites mobile café (sweetbitestruck)
    Swizzler (swizzlerfoods)
    Takorean (takorean)
    Tapas Truck (tapastruckdc)
    Tasha’s Cookies and Treats (tashascookies)
    Taste Of Eastern Europe (TasteOfEasternE)
    Tasty Fried DC (TastyFriedDC)
    Tasty Kabob (tastykabob)
    Tasty Toranj (TastyToranj)
    Tempo Di Pasta (TempoDiPasta)
    Thai Machine (thaimachine)
    That Cheesecake Truck! (cheesecaketruc)
    The Corn Factory (thecornfactory)
    The Frosted Muffin (tfmcupcakery)
    The Inside Scoop (tiscreamery)
    The Wagon (thewagonmeal)
    Tin Heaven (TinHeavenDC)
    Tokyo In The City (tokyointhecity)
    Top Dog (topdogtruck)
    Tops American Food Company (topstrucks)
    Tortuga (tortugatruck)
    TSR Quesadillas & Baked Empanadas (BakedTSR)
    Turkish Kabob (TURKISHKABOBDC)
    Urban Bumpkin (urbanbumpkinbbq)
    Urban Poutine (urbanpoutine)
    Village Cafe Express (VillageCafeINC)
    Wassub (wassubdc)
    Westray's Finest (WestraysFinest)
    What The Pho? (whatthephodc)
    Wi Jammin (wijammincaters)
    Woodland’s Vegan Bistro (WoodlandsVB)
    Yella Food (YellaFoodTruck)
    Yellow Vendor (yellowvendor)
    Yumm Yum Food (YummyYum_Food)
    Yumpling (yumplingdc)
    Z-Burger Mobile (ZBurgerMobile)
    Zesty Kabob (ZestyKabob)


# List of all handles
abunaifood
AmoriniPanini
AngryBurgerNOW
arepacrew
ArepaZone
AstroDoughnuts
Partytrucks
AZNeats
alocubanotruck
BabasBigBite
badabingdc
balkaniktaste
theballtruck
BanhMiAnnie
BasilThymeDC
bbqbusdc
gotbeachfries
BeirutDelights
bellavitawheels
bestburritos1
okbibija
bigcheesetruck
BITE2GO
eatBONMi
Borinquenlunchb
BLTfoodtruck
bratwurstking
bklynsandwichco
eatbrownbag
bubbleteaparty
burgorilla
CAslidercompany
cajunators
CapitalCW
capmacdc
CaptainCookieDC
Caribbean2Go
CarmensCarts
carnbbq
EatCarolinaQ
1styellowvendor
ChatpatTruck
cheesequakedc
chefalex2013
ChefonWheels11
chefsebonwheels
ChezAdilmos
chickfilamobile
ChittiGrill
ChixNStixDC
coles_palette
cornedbeefking
CrabCab
CRAVEIT2
cremedelacakedc
crepelovetruck
CrepesParfait
JoyCupcakes
CurbSideCrab
curbsidecupcake
CurleysQ
ThePieTruckDC
DcBallers
dc_doner
DCEmpanadas
dc_greekfood
dcpollos
DCQuesadilla
dcslices
DCSLIDERSTRUCK
DCTacoTruck
dirtysouthdeli
DolciGelatiTruk
donburidc
DorothyMoon1
dougthefooddude
DuckysGrub
elFuegoDc
EndlessPLLC
FarEastTG
Fasikacuisin
FavaPot
FedCityBros
feelincrabby
eatFireNRice
firemanscafe
FoodLady2011
foodforceone1
freshgreenfood
FroZenYoToGo
FCFoodTruck
GeorgesWingTruk
gofishtruckdc
mobilekitchen
GAHDTruck
GREENEGGBURGERS
GridsWaffles
GuaposFoodTruck
habebe19
wherehalal
hardysbbqdivine
healthyfool
henhousedc
HolyCrepesTruck
HotPeopleFood
DCHotWheels
HouseofFalafel
hulagirltruck
hungryheartllc
italiansubs1
JMCCurbside
jerkchickenfest
KababjiTruck
KabobBites
kabob_king
kabobpalace1
KaftaMania
kalaverasdc
KimchiBBQ
dckbbqbox
korengy
kravingkabob
kushimototruck
LATacoTruck
LaTingeriaTruck
latinamericanf1
lemongrasstruck
LilypadontheRun
Limetreetruck
LLuncheonette
LittleItalyDC
chefty99
loslobosburrito
macattackdc
macsdonuts
MakiShopTruck
MamasDonutBites
MayurKababHouse
Med_Delight
Meggrollmania
meskihealthy2go
mesobonwheels
Miamiviceburger
MidniteCC
MightyDogAcai
MisoHoneyTruck
mohmohdumpling
mojaitalatinfla
Mojotruck
Miyagifoodtruck
MuncheezDC
NaanStopDC
NeatMeatDC
NYDeliTruck
OliviasCupcakes
OohDaTChickeN
orangecowdc
overtherice
parskabob
passl0nfrult
pepefoodtruck
perubrothers
buritosontherun
philliesphamous
phojunkies
PhoBachiInc
dcphonation
PhoWheels
pleasantpops
PoppedRepublic
dcpuddin
quickwraps
therandyradish
RebasFunnelCake
LobstertruckDC
redbonefc
ReggaeVibesTruk
riochurrascoDC
RitoLoco
RoamingRoti
RocklandsTruck
RockSalt_
Rollinpizzadc
rollinkabob
RollsOnRolls
RovingItalian
asgharboa
saffrontruckdc
SangOnWheels
SaransVegiTruck
SateTruck
Scoops2u
senortaco7
SeoulFoodDC
sidewalksweets
simple_wheels
simplysoulfood
Sinplicity1
SloppyMamas
BayouGators
smokingkowbbq
solmexicangrill
SouthMeetsEast
SouvlakiStop
SpitfireTruck
steakbites
stellaspopkern
StreetCream34
SUNdeVICHtruck
surfsidetruckdc
SUSHIPAO
sweetbitestruck
swizzlerfoods
takorean
tapastruckdc
tashascookies
TasteOfEasternE
TastyFriedDC
tastykabob
TastyToranj
TempoDiPasta
thaimachine
cheesecaketruc
thecornfactory
tfmcupcakery
tiscreamery
thewagonmeal
TinHeavenDC
tokyointhecity
topdogtruck
topstrucks
tortugatruck
BakedTSR
TURKISHKABOBDC
urbanbumpkinbbq
urbanpoutine
VillageCafeINC
wassubdc
WestraysFinest
whatthephodc
wijammincaters
WoodlandsVB
YellaFoodTruck
yellowvendor
YummyYum_Food
yumplingdc
ZBurgerMobile
ZestyKabob